<a href="https://colab.research.google.com/github/theinem/FindMotif/blob/master/MotifsDetection_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Detección de Motifs V2**
Este notebook contiene los pasos para la proposición de motifs (motivos) candidatos dentro de un conjunto de cadenas de ADN.

Se analizará dormancy survival regulator (DosR). Este factor de transcripción regula la expresión de genes bajo condiciones hipóxicas para la turbeculosis micobacteriana. Se analizarán una serie de 15-mers para detectar los puntos de partida de los enlaces que disparan la actuación de DosR.

***La versión 2 emplea seudo-contadores para prevenir los errores donde aparecen probabilidades de cero***

Este método está inspirado en la aplicación de la ley de Cromwell y los seudoconteos de Laplace.

### Montar el disco virtual

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Lectura del archivo

In [2]:
#Alternativa no explorada por el curso. Cargar los datos desde el disco.
#f = open('/content/drive/My Drive/Colab Notebooks/Bioinformatics/Semana 3/DosR.txt', 'r')
#Dna = f.read()

Dna = ["GCGCCCCGCCCGGACAGCCATGCGCTAACCCTGGCTTCGATGGCGCCGGCTCAGTTAGGGCCGGAAGTCCCCAATGTGGCAGACCTTTCGCCCCTGGCGGACGAATGACCCCAGTGGCCGGGACTTCAGGCCCTATCGGAGGGCTCCGGCGCGGTGGTCGGATTTGTCTGTGGAGGTTACACCCCAATCGCAAGGATGCATTATGACCAGCGAGCTGAGCCTGGTCGCCACTGGAAAGGGGAGCAACATC", "CCGATCGGCATCACTATCGGTCCTGCGGCCGCCCATAGCGCTATATCCGGCTGGTGAAATCAATTGACAACCTTCGACTTTGAGGTGGCCTACGGCGAGGACAAGCCAGGCAAGCCAGCTGCCTCAACGCGCGCCAGTACGGGTCCATCGACCCGCGGCCCACGGGTCAAACGACCCTAGTGTTCGCTACGACGTGGTCGTACCTTCGGCAGCAGATCAGCAATAGCACCCCGACTCGAGGAGGATCCCG", "ACCGTCGATGTGCCCGGTCGCGCCGCGTCCACCTCGGTCATCGACCCCACGATGAGGACGCCATCGGCCGCGACCAAGCCCCGTGAAACTCTGACGGCGTGCTGGCCGGGCTGCGGCACCTGATCACCTTAGGGCACTTGGGCCACCACAACGGGCCGCCGGTCTCGACAGTGGCCACCACCACACAGGTGACTTCCGGCGGGACGTAAGTCCCTAACGCGTCGTTCCGCACGCGGTTAGCTTTGCTGCC", "GGGTCAGGTATATTTATCGCACACTTGGGCACATGACACACAAGCGCCAGAATCCCGGACCGAACCGAGCACCGTGGGTGGGCAGCCTCCATACAGCGATGACCTGATCGATCATCGGCCAGGGCGCCGGGCTTCCAACCGTGGCCGTCTCAGTACCCAGCCTCATTGACCCTTCGACGCATCCACTGCGCGTAAGTCGGCTCAACCCTTTCAAACCGCTGGATTACCGACCGCAGAAAGGGGGCAGGAC", "GTAGGTCAAACCGGGTGTACATACCCGCTCAATCGCCCAGCACTTCGGGCAGATCACCGGGTTTCCCCGGTATCACCAATACTGCCACCAAACACAGCAGGCGGGAAGGGGCGAAAGTCCCTTATCCGACAATAAAACTTCGCTTGTTCGACGCCCGGTTCACCCGATATGCACGGCGCCCAGCCATTCGTGACCGACGTCCCCAGCCCCAAGGCCGAACGACCCTAGGAGCCACGAGCAATTCACAGCG", "CCGCTGGCGACGCTGTTCGCCGGCAGCGTGCGTGACGACTTCGAGCTGCCCGACTACACCTGGTGACCACCGCCGACGGGCACCTCTCCGCCAGGTAGGCACGGTTTGTCGCCGGCAATGTGACCTTTGGGCGCGGTCTTGAGGACCTTCGGCCCCACCCACGAGGCCGCCGCCGGCCGATCGTATGACGTGCAATGTACGCCATAGGGTGCGTGTTACGGCGATTACCTGAAGGCGGCGGTGGTCCGGA", "GGCCAACTGCACCGCGCTCTTGATGACATCGGTGGTCACCATGGTGTCCGGCATGATCAACCTCCGCTGTTCGATATCACCCCGATCTTTCTGAACGGCGGTTGGCAGACAACAGGGTCAATGGTCCCCAAGTGGATCACCGACGGGCGCGGACAAATGGCCCGCGCTTCGGGGACTTCTGTCCCTAGCCCTGGCCACGATGGGCTGGTCGGATCAAAGGCATCCGTTTCCATCGATTAGGAGGCATCAA", "GTACATGTCCAGAGCGAGCCTCAGCTTCTGCGCAGCGACGGAAACTGCCACACTCAAAGCCTACTGGGCGCACGTGTGGCAACGAGTCGATCCACACGAAATGCCGCCGTTGGGCCGCGGACTAGCCGAATTTTCCGGGTGGTGACACAGCCCACATTTGGCATGGGACTTTCGGCCCTGTCCGCGTCCGTGTCGGCCAGACAAGCTTTGGGCATTGGCCACAATCGGGCCACAATCGAAAGCCGAGCAG", "GGCAGCTGTCGGCAACTGTAAGCCATTTCTGGGACTTTGCTGTGAAAAGCTGGGCGATGGTTGTGGACCTGGACGAGCCACCCGTGCGATAGGTGAGATTCATTCTCGCCCTGACGGGTTGCGTCTGTCATCGGTCGATAAGGACTAACGGCCCTCAGGTGGGGACCAACGCCCCTGGGAGATAGCGGTCCCCGCCAGTAACGTACCGCTGAACCGACGGGATGTATCCGCCCCAGCGAAGGAGACGGCG", "TCAGCACCATGACCGCCTGGCCACCAATCGCCCGTAACAAGCGGGACGTCCGCGACGACGCGTGCGCTAGCGCCGTGGCGGTGACAACGACCAGATATGGTCCGAGCACGCGGGCGAACCTCGTGTTCTGGCCTCGGCCAGTTGTGTAGAGCTCATCGCTGTCATCGAGCGATATCCGACCACTGATCCAAGTCGGGGGCTCTGGGGACCGAAGTCCCCGGGCTCGGAGCTATCGGACCTCACGATCACC"]

### Definimos todas las funciones
Nos permitirán hacer una greedy search

In [4]:
# Input:  A set of k-mers motifs
# Output: CountWithPseudocounts(Motifs)
def Count(Motifs):
    count = {} # initializing the count dictionary
    k = len(Motifs[0])
    count = {'A':[1]*k,'C':[1]*k,'G':[1]*k,'T':[1]*k}
    
    t = len(Motifs)
    for i in range(t):
        for j in range(k):
            symbol = Motifs[i][j]
            count[symbol][j] += 1
    return count

# Input:  A set of k-mers motifs
# Output: ProfileWithPseudocounts(Motifs)
def Profile(Motifs):
    t = len(Motifs)
    k = len(Motifs[0])
    profile = {} # output variable
    t = len(Motifs)+4
    k = len(Motifs[0])
    profile = Count(Motifs)
    for i in profile:  
        for j in range(k):
            profile[i][j] = profile[i][j]/t   
    return profile

# Input:  A set of k-mers motifs
# Output: A consensus string of Motifs.
def Consensus(Motifs):
    k = len(Motifs[0])
    count = Count(Motifs)
    consensus = ""
    for j in range(k):
        m = 0
        frequentSymbol = ""
        for symbol in "ACGT":
            if count[symbol][j] > m:
                m = count[symbol][j]
                frequentSymbol = symbol
        consensus += frequentSymbol
    return consensus


# Input:  A set of k-mers motifs
# Output: The score of these k-mers.
def Score(Motifs):
    result = Consensus(Motifs)
    counts = Count(Motifs)
    score = len(Motifs)*len(result)
    i = 0
    for symbol in result:
        # Score is the sum of (total number of elements per column MINUS
        # the number of occurence of the most frequent symbol per column)   
        score -= counts[symbol][i]
        i += 1
    return score


def Pr(Text, Profile):
    p = float(1)
    for i in range(len(Text)):
        p *= Profile[Text[i]][i]
    return p
# The profile matrix assumes that the first row corresponds to A, the second corresponds to C,
# the third corresponds to G, and the fourth corresponds to T.
# You should represent the profile matrix as a dictionary whose keys are 'A', 'C', 'G', and 'T' and whose values are lists of floats
def ProfileMostProbableKmer(text, k, profile):
    max_p = -1
    max_kmer = ''
    for i in range(len(text)-k+1):
        p = Pr(text[i:i+k], profile)
        if p > max_p:
            max_p = p
            max_kmer = text[i:i+k]            
    return max_kmer

# Input:  A list of k-mers Dna, and integers k and t (where t is the number of k-mers in Dna)
# Output: GreedyMotifSearch(Dna, k, t)
def GreedyMotifSearch(Dna, k, t):
    BestMotifs = []
    for i in range(0, t):
        BestMotifs.append(Dna[i][0:k])
    
    n = len(Dna[0])
    for i in range(n-k+1):
        Motifs = []
        Motifs.append(Dna[0][i:i+k])
        for j in range(1, t):
            P = Profile(Motifs[0:j])
            Motifs.append(ProfileMostProbableKmer(Dna[j], k, P))
    
        if Score(Motifs) < Score(BestMotifs):
            BestMotifs = Motifs
    return BestMotifs


### Ejecución de las funciones
Se buscará un 15-mer dentro de 10 segmentos de ADN.

In [5]:
t = len(Dna)
k = 15
Motifs = GreedyMotifSearch(Dna, k, t)

### Muestra de los motifs resultantes
En esta segunda versión volvemos a encontrar como primer nucleótido a la Guanina. Pero hay nuevos patrones encontrados como el de la Guanina en la segunda posición o la Citosina en la tercera. Este método muestra ser mucho más efectivo. Lo que se comprobará más adelante.

In [6]:
for i in range(len(Motifs)):
  print( i, '=', Motifs[i])

0 = GGACTTCAGGCCCTA
1 = GGTCAAACGACCCTA
2 = GGACGTAAGTCCCTA
3 = GGATTACCGACCGCA
4 = GGCCGAACGACCCTA
5 = GGACCTTCGGCCCCA
6 = GGACTTCTGTCCCTA
7 = GGACTTTCGGCCCTG
8 = GGACTAACGGCCCTC
9 = GGACCGAAGTCCCCG


### Consenso de los motifs y puntaje
El mejor puntaje posible es cero, es decir, se desea encontrar menor cantidad de diferencias entre los motifs propuestos. Para esta prueba se calcula un solo grupo de motifs, así que se obtiene una sola respuesta generada por el consenso de los motifs.

En este caso el resultado es 20, mientras que el método de la versión anterior encontraba un motif con score de 64. Una mejora de casi el 70%

In [7]:
print('Consenso:', Consensus(Motifs))
print('Puntaje:', Score(Motifs))

Consenso: GGACTTACGGCCCTA
Puntaje: 20


# **---Con esto damos por terminado el código---**

# *Limitaciones*

*   Al no hacer una búsqueda exhaustiva, puede que no encuentre la cadena óptima global.


Basado en los temas dictados en el curso Biology Meets Programming ( https://www.coursera.org/learn/bioinformatics/home/week/4 )

Otras fuentes: http://www.mrgraeme.co.uk/greedy-motif-search/